Retrieving (unstructured) greenery goals
---
This notebook retrieves a document with greenery goals in it. This document will be chuncked and then be stored in a vector database. At last it will do execute a similarity search to find relevant documents and it will then use these to find the answer for the question.


Imports
---

In [87]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.chains.retrieval_qa.base import RetrievalQA
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

from langchain_community.docstore import InMemoryDocstore

from langchain_openai import AzureChatOpenAI
from langchain_ollama import ChatOllama

from dotenv import load_dotenv
from PyPDF2 import PdfReader
from uuid import uuid4

import os
import faiss

Load a large language model
----------
Langchain makes it possible to easily switch LLMs. Llama 3 is used to show the data can be analysed with a locally running open-source model, but it is very slow. So to speed it up I also used o3-mini and gpt-4o-mini to show it works.

Load Llama3:


In [2]:
chosen_llm = ChatOllama(base_url='http://localhost:11434', model="llama3")

Load o3-mini (via Azure):

In [3]:
load_dotenv()

chosen_llm = AzureChatOpenAI(model ="o3-mini", api_version="2025-01-01-preview", azure_endpoint="https://56948-m9bdjgpg-eastus2.cognitiveservices.azure.com/openai/deployments/o3-mini/chat/completions?api-version=2025-01-01-preview", api_key=os.environ.get("AZURE_OPENAI_API_KEY"))

Load gpt-4o-mini (via Azure)

In [3]:
load_dotenv()

chosen_llm = AzureChatOpenAI(model="gpt-4o-mini", api_version="2025-01-01-preview",
                             azure_endpoint="https://56948-m9bdjgpg-eastus2.cognitiveservices.azure.com/openai/deployments/gpt-4o-mini/chat/completions?api-version=2025-01-01-preview",
                             api_key=os.environ.get("AZURE_OPENAI_API_KEY"))

Define the question
---
Write the question that needs to be answered by the LLM.

In [88]:
query = "Hoeveel straattegels moeten er vevangen worden door groen?"

Load and chunk the document
---
It loads the pdf, extracts the text and chunks it into pieces.

In [61]:
# Load file
file_path = "./GroenvisieSchiedam.pdf"
doc_reader = PdfReader(file_path)

# Extract page content
raw_text = ''
for i, page in enumerate(doc_reader.pages):
    text = page.extract_text()
    if text:
        raw_text += text

print("raw text characters: " + str(len(raw_text)))

# Split the page content into chunks
text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 833,
    chunk_overlap  = 200,
    length_function = len,
)
chunks = text_splitter.split_text(raw_text)

print("chunk text characters: " + str(len(chunks)))


raw text characters: 9039
chunk text characters: 15


Generate embeddings and save them in the vector database
---
Create a vector database, generate embeddings for the chunks and save the embeddings in the newly created vector database.

In [64]:
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
index = faiss.IndexFlatL2(len(embedding_model.embed_query(chunks[0]))) # Calculates the amount of dimensions the chunk's vector has

vector_store = FAISS(
    embedding_function=embedding_model,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

uuids = []
for chunk in range(len(chunks)):
    uuids.append(str(uuid4()))

vector_store.add_texts(texts=chunks, ids=uuids)

print("UUIDs of items stored in vector database: " + str(uuids))


UUIDs of items stored in vector database: ['052fb115-8b29-4a49-b135-d5f1eb8887ce', '5589ffd4-8217-4fc9-a060-bc5f37e195bf', '3104cef4-5191-42ad-8a33-22e0a5c29814', '9580a592-5f27-42ae-bf5b-be6b3635fa80', 'd2d4d3eb-4197-4bee-bb3d-7cdc9888b1d3', '87a38a9e-5b29-4346-a4c3-4b783f504311', 'f7a5f8de-1233-43e3-9944-a28cca6243b5', '0e287dc4-c284-42ce-943a-5b4d3fbaaf5e', '33c05eb9-4671-44da-8edf-cd21860f93e8', '0faa5e74-232e-42d1-a9ca-2a435e6364d4', 'e9d0e2e7-5df1-449b-8e8f-50d485aa2394', '830c7cc7-93c9-4e12-ba71-6e3e14c19bd2', 'ea3ff7da-ea55-4933-be54-0c2d18c6a9e2', 'ab855ffc-34fa-4b4b-9f4a-72db995d3855', 'e9e85cbc-e7d8-47b3-b04b-63e6756c8548']


Gather relevant documents
---
Execute a similarity search between the query and the vector database to find the 2 most relevant documents.

In [81]:
relevant_documents = vector_store.similarity_search(query, k=2)

print(f"Vraag: {query}")
print(f"Relevante documenten: ")

for document in relevant_documents:
    print(f" \n-->   {document.page_content} \n")

Vraag: Hoeveel straattegels moeten er vevangen worden door groen?
Relevante documenten: 
 
-->   We ontwikkelen en/of versterken verbindingen tussen groene gebieden door zoveel mogelijk onverharde paden aan te leggen die kunnen worden gebruikt door wandelaars, hondenbezitters en mountainbikers. 
De komende jaren maken we voor alle parken – de groene parels – samen met bewoners beheerplannen. 
Per wijk brengen we belangrijke plekken in kaart, doen we voorstellen ter verbetering en laten we zien hoe we deze kunnen realiseren.  Publieksversie Groenvisie  |  Gemeente Schiedam   
    
    
 
 9 
Visie op toekomst en klimaat 5 
Minimaal 5% van de straattegels/klinkers willen we vervangen door groen. 
Het gaat om ongeveer 10.000 m2. 
Door directe infiltratie in het groen vergroten we de waterberging in de stad. We stimuleren inwoners om ook regenwater af te koppelen en direct in de eigen tuin te laten infiltreren. 

 
-->   Bomen en beplanting hebben voldoende ruimte en goede omstandigheden o

Analyze the relevant documents and answer the question
---
The last step is to create a question and answer chain where the chosen llm can actually answer the question.

In [86]:
qa_chain = RetrievalQA.from_chain_type(
    llm=chosen_llm,
    chain_type="stuff",  # of "map_reduce" bij grote documenten
    retriever=vector_store.as_retriever(search_kwargs={"k": 2}),
    return_source_documents=True
)

result = qa_chain(query)

print("Vraag:", query)
print("Antwoord:", result['result'])
print("Bronnen: \n")
for doc in result['source_documents']:
    print(doc.page_content)

Vraag: Hoeveel straattegels moeten er vevangen worden door groen?
Antwoord: Minimaal 5% van de straattegels/klinkers moeten vervangen worden door groen, wat ongeveer 10.000 m2 betreft.
Bronnen: 

We ontwikkelen en/of versterken verbindingen tussen groene gebieden door zoveel mogelijk onverharde paden aan te leggen die kunnen worden gebruikt door wandelaars, hondenbezitters en mountainbikers. 
De komende jaren maken we voor alle parken – de groene parels – samen met bewoners beheerplannen. 
Per wijk brengen we belangrijke plekken in kaart, doen we voorstellen ter verbetering en laten we zien hoe we deze kunnen realiseren.  Publieksversie Groenvisie  |  Gemeente Schiedam   
    
    
 
 9 
Visie op toekomst en klimaat 5 
Minimaal 5% van de straattegels/klinkers willen we vervangen door groen. 
Het gaat om ongeveer 10.000 m2. 
Door directe infiltratie in het groen vergroten we de waterberging in de stad. We stimuleren inwoners om ook regenwater af te koppelen en direct in de eigen tuin te